# Using information from GeoTOP
Most geohydrological models in the Netherlands use the layer model REGIS as the basis for the geohydrological properties of the model. However, REGIS does not contain information for all layers, of which the holocene confining layer (HLc) is the most important one. In this notebook we will show how to use the voxel model GeoTOP to generate geohydrolocal properties for this layer.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import LineString

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

In [ ]:
# Define an extent and a line from southwest to northeast through this extent
extent = [100000.0, 105000.0, 499800.0, 500000.0]
line = LineString([(extent[0], extent[2]), (extent[1], extent[3])])

We define a method called 'plot_kh_kv', which we can call in cells below to plot the horizontal and vertical conductivity in a cross-section.

In [ ]:
def plot_kh_kv(
    ds,
    layer="layer",
    variables=None,
    zmin=-50.25,
    min_label_area=None,
    cmap=None,
    norm=None,
):
    if variables is None:
        variables = ["kh", "kv"]
    if cmap is None:
        cmap = plt.get_cmap("turbo_r")
    if norm is None:
        boundaries = [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100]
        norm = matplotlib.colors.BoundaryNorm(boundaries, cmap.N)
    for var in variables:
        f, ax = plt.subplots(figsize=(10, 5))
        cs = nlmod.plot.DatasetCrossSection(ds, line, layer=layer, ax=ax, zmin=zmin)
        pc = cs.plot_array(ds[var], norm=norm, cmap=cmap)
        if min_label_area is not None:
            cs.plot_layers(alpha=0.0, min_label_area=min_label_area)
            cs.plot_grid(vertical=False)
        format = matplotlib.ticker.FuncFormatter(lambda y, _: "{:g}".format(y))
        nlmod.plot.colorbar_inside(pc, bounds=[0.05, 0.05, 0.02, 0.9], format=format)
        nlmod.plot.title_inside(var, ax=ax)
        ax.set_xlabel("afstand langs doorsnede (m)")
        ax.set_ylabel("z (m NAP)")
        f.tight_layout(pad=0.0)

## Download GeoTOP
We download GeoTOP for a certain extent. We get an xarray.Dataset with voxels of 100 * 100 * 0.5 (dx * dy * dz) m, with variables 'strat' and 'lithok'. We also get the probaliblity of the occurence of each lithoclass in variables named 'kans_*' (since we set `probabilities=True`).

In [ ]:
gt = nlmod.read.geotop.get_geotop(extent, probabilities=True)
gt

We plot the lithoclass (soil types) in a cross-section using the method `nlmod.plot.geotop_lithok_in_cross_section`.

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
nlmod.plot.geotop_lithok_in_cross_section(line, gt)
f.tight_layout(pad=0.0)

## Add hydrological properties (kh and kv)
GeoTOP does not contain information about geohydroloical propties directly. We need to calculate this information, using the lithoclass, and optionally the stratigraphy (layer unit). We get this information from a DataFrame, whch needs to contain the columns 'lithok' and 'kh' (and optionally 'strat' and 'kv').

### Based on lithok
With `nlmod.read.geotop.get_lithok_props()` we get a default value for each of the 9 lithoclasses (lithoclass 4 is not used). These values are a rough estimate of the hydrologic conductivity. We recommend changing these values based on local conditions.

In [ ]:
df = nlmod.read.geotop.get_lithok_props()
df

The method `nlmod.read.geotop.add_kh_and_kv` takes this DataFrame, applies it to the GeoTOP voxel-dataset `gt`, and adds the variables `kh` and `kv` to `gt`.

In [ ]:
gt = nlmod.read.geotop.add_kh_and_kv(gt, df)

When we plot kh and kv we see fine sands get a value of 5 m/d (green) and medium fine sands get a value of 10 m/d (light blue). We see the peat (0.001 m/d) and clay (0.01 m/d) layers as zones with lower conductivities.

In [ ]:
plot_kh_kv(gt, layer="z")

### Based on lithok and strat
We can also load one of the other  DataFrames that are built into nlmod, using the method `nlmod.geotop.get_kh_kv_table()`. Using this method, a table for certain location can be loaded. Right now, the only allowed value for `kind` is 'Brabant', which is the default value, and loads the hydrological properties per lithoclass and stratigraphic unit.

In [ ]:
df = nlmod.read.geotop.get_kh_kv_table(kind="Brabant")
df

We use this table to add a kh and kv value for each voxel, in variables named 'kh' and 'kv'.

In [ ]:
gt = nlmod.read.geotop.add_kh_and_kv(gt, df)
gt

We can plot these values along the same line we plotted the lithoclass-values in.

In [ ]:
plot_kh_kv(gt, layer="z")

## Aggregate voxels to GeoTOP-layers
In the previous example we have added geodrological properties to the voxels of GeoTOP and plotted them. The layers of a groundwatermodel generally are thicker than the thickness of the voxels (0.5 m). Therefore we need to aggregate the voxel-data into the layers of the model. We show this process by using the stratigraphy-data of GeoTOP to form a layer model, using the method `nlmod.read.geotop.to_model_layers`.

When values for `kh` and `kv` are present in `gt`, this method also calculates the geohydrological properties of the layer model with the method `nlmod.read.geotop.aggregate_to_ds`. The method calculates the combined horizontal transmissivity, and the combined vertical resistance of all (parts of) voxels in a layer, and calculates a `kh` and `kv` value from this transmissivity and resistance.

In [ ]:
gtl = nlmod.read.geotop.to_model_layers(gt)
gtl

We can plot the kh and kv value for each of the layers with the same method we used for the voxel-data.

In [ ]:
plot_kh_kv(gtl, min_label_area=1000.0)

## Aggregate voxels to REGIS-layers
We can use any layer model in `nlmod.read.geotop.aggregate_to_ds()`, also one from REGIS. Let's demonstrate this by downloading REGIS-data for the same extent.

In [ ]:
regis = nlmod.read.get_regis(extent)
regis

First we plot the original hydrological properties of REGIS. We see that kh is defined for the aquifers (top plot) and kv is defined for the aquitards (bottom plot). Neither kh or kv is defined for the top layer HLc.

In [ ]:
plot_kh_kv(regis, min_label_area=1000.0, zmin=-100.0)

We will add varibales `kh_gt` and `kv_gt` that contain the kh- and kv-values calculated from GeoTOP. Layers that do not contain any voxel will get a NaN-value for kh and kv.

In [ ]:
# make sure there are no NaNs in top and botm of layers
regis = nlmod.read.geotop.aggregate_to_ds(gt, regis, kh="kh_gt", kv="kv_gt")

When we plot the kh and kv value, we see all layers above -50 m NAP now contain values, calculated from the GeoTOP-data.

In [ ]:
plot_kh_kv(regis, min_label_area=1000.0, zmin=-100.0, variables=["kh_gt", "kv_gt"])

We can plot kh of one of the layers on a map, for both REGIS and GeoTOP. We generally see that conductivities in GeoTOP are a bit lower.

In [ ]:
layer = "KRz3"
var = "kh"
norm = matplotlib.colors.Normalize(0.0, 40.0)

f, axes = nlmod.plot.get_map(extent, nrows=2, figsize=(16, 8))
pc = nlmod.plot.data_array(regis[var].loc[layer], ax=axes[0], norm=norm)
nlmod.plot.colorbar_inside(pc, bounds=[0.02, 0.05, 0.02, 0.9], ax=axes[0])
nlmod.plot.title_inside("REGIS", ax=axes[0])
pc = nlmod.plot.data_array(regis[f"{var}_gt"].loc[layer], ax=axes[1], norm=norm)
nlmod.plot.title_inside("GeoTOP", ax=axes[1])
nlmod.plot.colorbar_inside(pc, bounds=[0.02, 0.05, 0.02, 0.9], ax=axes[1]);

## Using stochastic data from GeoTOP
In the previous section we used the most likely values from the lithoclass data of GeoTOP. GeoTOP is constructed by generating 100 realisations of this data. Using these realisations a probablity is determined for the occurence in each pixel for each of the lithoclassses. We can also use these probabilities to determine the kh and kv-value of each voxel. We do this by settting the `stochastic` parameter in `nlmod.read.geotop.add_kh_and_kv` to True. The kh and kv values are now calculated by a weighted average of the lithoclass data in each voxel, where the weights are determined by the probablilities. By default an arithmetic weighted mean is used for kh and a harmonic weighted mean for kv, but these methods can be chosen by the user.

In [ ]:
gt = nlmod.read.geotop.add_kh_and_kv(gt, df, stochastic=True)

We can plot the kh- and kv-values again. Using the stochastic data generally results in smoother values for kh and kv.

In [ ]:
plot_kh_kv(gt, layer="z")